In [0]:
## https://qiita.com/cometscome_phys/items/a31ab8a4a4f7217a70ff

## install manually
#!git clone https://github.com/QEF/q-e.git
#!apt-get install -y libfftw3-3 libfftw3-dev libfftw3-doc
#%cd q-e
#!DFLAGS='-D__OPENMP -D__FFTW3 -D__MPI -D__SCALAPACK' FFT_LIBS='-lfftw3'  ./configure --enable-openmp
#!make pw
#!make pp
#
## save binary files
#from google.colab import drive
#drive.mount('/content/drive')
#%cd /content/
#!zip -r /content/drive/'My Drive'/q-e.zip q-e

## install via MateriApps LIVE!
!wget https://sf.net/projects/materiappslive/files/Debian/sources/setup.sh
!sh setup.sh
!apt-get install quantum-espresso

#----

## run Quantum ESPRESSO
import os
os.environ['PATH'] = "/content/q-e/bin:"+os.environ['PATH']
#----
!pip install ase
#----
%cd /content
!mkdir NaCl
%cd NaCl
#----
!wget https://www.quantum-espresso.org/upf_files/Na.pbesol-spn-kjpaw_psl.1.0.0.UPF
!wget https://www.quantum-espresso.org/upf_files/Cl.pbesol-n-kjpaw_psl.1.0.0.UPF
#----
from ase.build import bulk
from ase.calculators.espresso import Espresso
from ase.constraints import UnitCellFilter
from ase.optimize import LBFGS
import ase.io 
pseudopotentials = {'Na': 'Na.pbesol-spn-kjpaw_psl.1.0.0.UPF',
                    'Cl': 'Cl.pbesol-n-kjpaw_psl.1.0.0.UPF'}  
rocksalt = bulk('NaCl', crystalstructure='rocksalt', a=6.0)
calc = Espresso(pseudopotentials=pseudopotentials,pseudo_dir = './',
                tstress=True, tprnfor=True, kpts=(3, 3, 3))
rocksalt.set_calculator(calc)
ucf = UnitCellFilter(rocksalt)
opt = LBFGS(ucf)
opt.run(fmax=0.005)
# cubic lattic constant
print((8*rocksalt.get_volume()/len(rocksalt))**(1.0/3.0))
#----
%cd /content
!mkdir Cu
%cd Cu
#----
!wget https://www.quantum-espresso.org/upf_files/Cu.pz-d-rrkjus.UPF
#----
from ase import Atoms
from ase.build import bulk
from ase.calculators.espresso import Espresso
atoms = bulk("Cu")
pseudopotentials = {'Cu':'Cu.pz-d-rrkjus.UPF'}
input_data = {
    'system': {
        'ecutwfc': 30,
        'ecutrho': 240,
        'nbnd' : 35,
    'occupations' : 'smearing',
        'smearing':'gauss',
        'degauss' : 0.01},
    'disk_io': 'low'}  # automatically put into 'control'
calc = Espresso(pseudopotentials=pseudopotentials,kpts=(4, 4, 4),input_data=input_data,pseudo_dir = './')
atoms.set_calculator(calc)
atoms.get_potential_energy()
fermi_level = calc.get_fermi_level()
print(fermi_level)
#----
input_data.update({'calculation':'bands',
                              'restart_mode':'restart',
                               'verbosity':'high'})
calc.set(kpts={'path':'GXWLGK', 'npoints':100},
          input_data=input_data)
calc.calculate(atoms)
#----
import matplotlib.pyplot as plt
bs = calc.band_structure()
bs.reference = fermi_level
bs.plot(emax=40,emin=5)